In [1]:
import os
import pathlib
from urllib.request import urlretrieve
from time import sleep
import math as math
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as ticker
from datetime import date
import boto3

import sys
sys.path.append('/code')
from fdmonitor.cont_futures import create_continuous_future, get_cme_data
from fdmonitor.plotter import draw_charts

%matplotlib inline

In [2]:
cme_metadata = pd.read_csv('/code/data/CME_metadata.csv')
cme_futures = pd.read_csv('/code/data/CME_futures.csv')
cme_futures.set_index('Futures Symbol', inplace=True)
DATA_DIRNAME = pathlib.Path('/code').resolve() / 'data'

In [4]:
futures_symbol = 'ES'
futures_name, CME_months, CME_years = get_cme_data(futures_symbol, cme_futures, cme_metadata)

In [5]:
data = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                                        'open_interest', False)

In [5]:
def create_pdf(futures_symbol, cme_futures, cme_metadata):
    futures_name, CME_months, CME_years = get_cme_data(futures_symbol, 
                                                       cme_futures, cme_metadata)
    roll_rules = ['expiration', 'open_interest', 'exp_month']
    true_false = [False, True]

    pdf_pages = PdfPages(pathlib.Path('/code/pdfs/tmp').resolve() / f'{futures_symbol}.pdf')

    for roll_rule in roll_rules:
        for tf in true_false:
            data = create_continuous_future(futures_symbol, DATA_DIRNAME, CME_months, CME_years,
                                        roll_rule, tf)
            plot_title = f'{futures_name} ({futures_symbol}) \n Roll: {roll_rule}'
            if tf:
                plot_title = plot_title + ', backadjusted'
            draw_charts(data, plot_title, pdf_pages)
    pdf_pages.close()

In [6]:
create_pdf('NQ', cme_futures, cme_metadata)

In [7]:
for futures_symbol in cme_futures.index.tolist():
    create_pdf(futures_symbol, cme_futures, cme_metadata)

/usr/local/lib/python3.6/site-packages/matplotlib/tight_layout.py:199: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  warnings.warn('Tight layout not applied. '


In [9]:
data.index[-1].strftime('%m_%d_%y')

'11_29_18'

In [4]:
def create_combined_pdf(cme_futures, cme_metadata):
    today = date.today().isoformat()
    pdf_pages = PdfPages(f'/code/pdfs/{today}.pdf')
    futures_symbols = cme_futures.index.tolist()
    for futures_symbol in futures_symbols:
        futures_name, CME_months, CME_years = get_cme_data(futures_symbol, 
                                                       cme_futures, cme_metadata)
        data = create_continuous_future(futures_symbol, CME_months, CME_years,
                                        'open_interest', is_backadjusted=False, 
                                        use_s3=True)
        date_string = data.index[-1].strftime('%m/%-d/%y')
        plot_title = f'{futures_name} ({futures_symbol})\n{date_string}\nRoll: open_interest'
        draw_charts(data, plot_title, pdf_pages)
    pdf_pages.close()
    
create_combined_pdf(cme_futures, cme_metadata)

In [18]:
today = date.today().isoformat()
print(today)

2018-12-07


In [22]:
futures_code = 'ES'
contract_code = 'ESZ2018'
s3_filename = f'data/raw_csv/futures/{futures_code}/daily/{contract_code}.csv'
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='financial-data-monitor', Key=s3_filename)
current_df = pd.read_csv(obj['Body'])
current_df.tail()

,Date,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
307,2018-11-30,2742.00,2764.75,2729.00,2764.00,14.00,2758.25,1418344.0,2749286.0
308,2018-12-03,2793.50,2814.00,2773.50,2789.75,32.50,2790.75,1842048.0,2737306.0
309,2018-12-04,2789.75,2790.50,2696.00,2701.75,89.00,2701.75,2617933.0,2740857.0
310,2018-12-05,2699.50,2720.75,2698.00,2718.00,NaN,2701.75,184485.0,2746850.0
311,2018-12-06,2713.25,2714.00,2621.25,2696.25,10.75,2691.00,3240785.0,2742552.0
